In [2]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, Model
from keras.layers import Dropout, Flatten, Dense, Input
from keras import applications
from keras.layers import AveragePooling2D, GlobalAveragePooling2D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping, CSVLogger, TensorBoard
from keras.applications.inception_resnet_v2 import InceptionResNetV2, preprocess_input 
from keras.layers import Conv2D, Convolution2D, MaxPooling2D, ZeroPadding2D, BatchNormalization, Activation
from keras.optimizers import Adam
from keras import backend as K
import numpy as np
import pandas as pd
from keras import layers
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from keras.models import load_model
import random
import cv2
from PIL import Image
import os
from keras.utils import to_categorical
from collections import defaultdict

In [3]:
#parameters
classes = 10
batch_size= 60
image_size = 224
train_total = 10000
validation_total = 4370

In [4]:
def CACNN_basic_block(x):
    x = Conv2D(8, (3,3), padding='valid')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = MaxPooling2D((2,2))(x)
    
    x = Conv2D(16, (3,3), padding='valid')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = MaxPooling2D((2,2))(x)
    
    x = Conv2D(32, (3,3), padding='valid')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = MaxPooling2D((2,2))(x)
    
    x = Conv2D(64, (3,3), padding='valid')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = MaxPooling2D((2,2))(x)
    
    x = Conv2D(128, (3,3), padding='valid')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = MaxPooling2D((2,2))(x)
    
    x = GlobalAveragePooling2D()(x)
    return x

In [5]:
x = Input((image_size,image_size,3))

x1 = Conv2D(4, (3,3), padding='valid')(x)
x1 = BatchNormalization()(x1)
x1 = CACNN_basic_block(x1)

x2 = Conv2D(4, (5,5), padding='valid')(x)
x2 = BatchNormalization()(x2)
x2 = CACNN_basic_block(x2)

x3 = Conv2D(4, (7,7), padding='valid')(x)
x3 = BatchNormalization()(x3)
x3 = CACNN_basic_block(x3)

y = layers.concatenate([x1,x2,x3])
#y = Dropout(0.3)(y)
y = Dense(256, activation='relu')(y)
y = Dropout(0.5)(y)
y = BatchNormalization()(y)
y = Dense(128, activation='relu')(y)
y = Dropout(0.5)(y)
y = BatchNormalization()(y)
y = Dense(classes, activation='softmax')(y)

In [6]:
model  = Model(inputs=x,outputs=y)
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 222, 222, 4)  112         input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_7 (Conv2D)               (None, 220, 220, 4)  304         input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_13 (Conv2D)              (None, 218, 218, 4)  592         input_1[0][0]                    
__________________________________________________________________________________________________
batch_norm

In [7]:
for layer in model.layers[:-7]:
    layer.trainable = True

In [8]:
train_datagen = ImageDataGenerator(horizontal_flip=True)
validation_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
                    

In [27]:
train_generator = train_datagen.flow_from_directory(
                    'train',
                    target_size=(image_size,image_size),
                    batch_size=batch_size,
                    class_mode='categorical',
                    shuffle=True
                    )

Found 9081 images belonging to 10 classes.


In [8]:
validation_generator = validation_datagen.flow_from_directory(
                        '../validation_224/',
                        target_size=(image_size,image_size),
                        batch_size=batch_size,
                        class_mode='categorical',
                        shuffle=False)

Found 4370 images belonging to 10 classes.


In [12]:
def random_crop(image, crop_size):
    h,w,d = image.shape
    rand_num_h = random.randint(0,h-crop_size)
    rand_num_w = random.randint(0,w-crop_size)
    image_crop = image[rand_num_h:rand_num_h+crop_size,rand_num_w:rand_num_w+crop_size,:]
    return image_crop

In [13]:
def random_crop_valid(image, crop_size):
    h,w,d = image.shape
    image_crop = image[h//2-crop_size//2:h//2+crop_size//2,w//2-crop_size//2:w//2+crop_size//2,:]
    
    return image_crop

In [34]:
def train_gen(train_files, train_classes, batch_size, target_size, imggen):
    min_batch_size = batch_size //10
    train_data = list(zip(train_files, train_classes))
    train_data_dict = defaultdict(list)
    for i in range(len(train_data)):
        train_data_dict[train_data[i][0].split('/')[0]].append(train_data[i])
       
    while(True):
        for i in train_data_dict.keys():
            random.shuffle(train_data_dict[i])
        for start in range(0, len(train_data), min_batch_size):
            image_crop_list = []
            image_classes_list = []
            for j in train_data_dict.keys():
                end = min(start + min_batch_size, len(train_data_dict[j])) 
    
                for i in range(start,end):
                    image_classes_list.append(train_data_dict[j][i][1])
                
                x_batch = [train_data_dict[j][i][0] for i in range(start,end)]
                for i in x_batch:
                    image = mpimg.imread('train/' + i)
                    image = random_crop(image,image_size)
                    if(np.random.rand()<0.5):
                        image = np.rot90(image,1,(0,1))
                    image_crop_list.append(image)
                    del image
                    
            
            if(len(image_classes_list)<batch_size-2*min_batch_size):
                break
            
            image_crop_list = [x/1. for x in image_crop_list]
            image_crop_list = [imggen.random_transform(x) for x in image_crop_list]
            image_crop_list = [preprocess_input(x) for x in image_crop_list]
            
            x_batch = np.array(image_crop_list, np.float32)
            y_batch = np.array(image_classes_list)         
            
            yield (x_batch, y_batch)

In [ ]:
train_crop_generator = train_gen(train_generator.filenames,to_categorical(train_generator.classes),batch_size,image_size,train_datagen) 

In [15]:
def valid_gen(valid_files, valid_classes, batch_size, target_size, imggen):

    valid_data = list(zip(valid_files, valid_classes))
  
    while(True):
        
        for start in range(0, len(valid_data), batch_size):
            image_crop_list = []
            image_classes_list = []
            end = min(start + batch_size, len(valid_data)) 
    
            x_batch = [valid_data[i][0] for i in range(start,end)]
            
            for index,i in enumerate(x_batch):
                image = mpimg.imread('../validation_224/' + i)
                if(len(image.shape)<3):
                        continue
                #rand_num = random.randint(0,7)
                
                #augmented_image = image_aug_valid(image,rand_num)
                #image = random_crop_valid(image,image_size)
                image_crop_list.append(image)
                image_classes_list.append(valid_data[start+index][1])
                
                del image
               
                #del aug_image_recrop
            
            
            image_crop_list = [x/1. for x in image_crop_list]
            #image_crop_list = [imggen.random_transform(x) for x in image_crop_list]
            image_crop_list = [preprocess_input(x) for x in image_crop_list]
            
            x_batch = np.array(image_crop_list, np.float32)
            y_batch = np.array(image_classes_list)         
            
            yield (x_batch, y_batch)

In [16]:
valid_aug_generator = valid_gen(validation_generator.filenames,to_categorical(validation_generator.classes),batch_size,image_size,validation_datagen) 

In [9]:
model.compile(loss='categorical_crossentropy',
             optimizer=Adam(lr=2.5e-5),
             metrics=['accuracy'])

In [38]:
callbacks = [ModelCheckpoint(filepath='cacnn_network_2_4.hdf5', verbose=1, save_best_only=True, save_weights_only=True),
ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1, epsilon=0.01),
EarlyStopping(monitor='val_loss', patience=10, verbose=1),
CSVLogger('./4-metrics_1.csv'),
TensorBoard(log_dir='logs_cacnn', write_graph=True)]

In [39]:
model.fit_generator(
                    train_crop_generator,
                    steps_per_epoch = 5*int(np.ceil(train_total/batch_size)),
                    epochs=100,
                    verbose=1,
                    validation_data=valid_aug_generator,
                    validation_steps= int(np.ceil(validation_total/batch_size)),
                    callbacks=callbacks)

NameError: name 'train_crop_generator' is not defined

In [24]:
model.load_weights('cacnn_network_2_2.hdf5')

In [25]:
model.evaluate_generator(valid_aug_generator,int(np.ceil(validation_total/batch_size)))

[0.34162709475265818, 0.91189931977531735]

In [1]:
# validation predictions for futture ensembling

In [10]:
model.load_weights('cacnn_network_2_3.hdf5')

In [41]:
cacnn_preds = model.predict_generator(valid_aug_generator,int(np.ceil(validation_total/batch_size)))

In [42]:
validation_filenames = np.load('filenames_list.npy')
f = defaultdict(list)
for i,j in enumerate(validation_generator.filenames):
    f[j.split('/')[-1][2:]].append(i)
    

In [43]:
result_filenames=[]
predicted_classes=[]
for i in validation_filenames:
    x = np.mean(cacnn_preds[f[i.split('/')[-1]]],axis=0)
    result_filenames.append(i)
    predicted_classes.append(x)

In [45]:
np.save('cacnn_preds_224.npy',predicted_classes)

In [ ]:
np.save('cacnn_preds.npy',cacnn_preds)

In [4]:
#test predicitons - 224 size

In [11]:
filenames_list = []
images_list = []
for i in os.listdir('test_224/temp/'):
    img = Image.open('test_224/temp/'+i)
    filenames_list.append(i)
    images_list.append(np.array(img, np.float32))
    

In [12]:
images_list = [preprocess_input(x) for x in images_list]
new_array_arr = np.array(images_list)
new_array_arr.shape

(13200, 224, 224, 3)

In [13]:
test_preds_224 = model.predict(new_array_arr)

In [14]:
f = defaultdict(list)
for i,j in enumerate(filenames_list):
    f[j[2:]].append(i)

In [15]:
test_filenames = np.load('test_filenames.npy')

In [16]:
result_filenames=[]
predicted_classes=[]
for i in test_filenames:
    x = np.mean(test_preds_224[f[i.split('/')[-1]]],axis=0)
    result_filenames.append(i)
    predicted_classes.append(x)

In [17]:
np.save('cacnn_test_preds_224.npy',predicted_classes)

In [13]:
filenames_list = []

In [3]:
# test predictions - 512 size

In [19]:
images_list = []
for i in os.listdir('test/temp/')[1000:]:
    img = Image.open('test/temp/'+i)
    filenames_list.append(i)
    images_list.append(np.array(img, np.float32))
    del img

In [20]:
images_list = [x/1. for x in images_list]

In [21]:
images_list = [preprocess_input(x) for x in images_list]

In [22]:
images_list = np.array(images_list)

In [18]:
test_preds = model.predict(images_list)

In [23]:
test_preds1 = model.predict(images_list)

In [20]:
test_filenames = np.load('test_filenames.npy')

In [24]:
final_test_preds = np.concatenate([test_preds,test_preds1],axis=0)

In [25]:
np.save('cacnn_test_preds.npy',final_test_preds)

In [56]:
class_ids = {train_generator.class_indices[x]: x for x in train_generator.class_indices}
predicted_classes = [class_ids[x] for x in np.argmax(test_preds, axis=1)]

In [63]:
for index,i in enumerate(filenames_list):
    if(i.split('_')[-1]=='manip.tif'):
        predicted_classes[index]='HTC-1-M7'

In [64]:
submission = pd.DataFrame({'fname':filenames_list,'camera':predicted_classes})
submission.to_csv('submission_mobilenet_v2_test.csv', encoding="utf8", index=False)

In [65]:
from IPython.display import FileLink
FileLink('submission_mobilenet_v2_test.csv')

/home/sainath/camera_model/submission_mobilenet_v2_test.csv